In [ ]:
!cp /content/drive/MyDrive/Uva/plotly_utils.py ./custom_plotly_utils.py
#import plotly_utils as pu
from custom_plotly_utils import imshow, line, scatter

In [ ]:
#imports

%load_ext autoreload
%autoreload 2

from diff_directions import *
from transformer_lens import HookedTransformer
import torch
from random import sample
import pandas as pd


In [ ]:
#model
model = HookedTransformer.from_pretrained(
    "gpt2-small",
    center_unembed=True,
    center_writing_weights=True,
    fold_ln=True,
    refactor_factored_attn_matrices=True
)

In [ ]:
#dataset 

df = pd.read_csv("/content/toxic_duplicated_max20namespercntry_wikinames_llama3.csv")
df = df[~df['Localized Name'].isna()].reset_index(drop=True)
# df = df.sample(128).reset_index(drop=True)
df_new = df.sample(n=10).reset_index(drop=True)

sentences = df_new["new_sentence"].to_list()
toxicity = df_new["toxicity"].to_list()

prompt = """Given a STATEMENT about what a person writes in chat, answer 'true' or 'false' for whether their statement is toxic
Template: "[[STATEMENT]]", what the person wrote is toxic: [[true]] or [[false]]
Example 1: "John wrote in the chat, 'hi how are you'", what the person wrote is toxic: false
Example 2: "John wrote in the chat, 'bitch you are shit'", what the person wrote is toxic: true
Input: "{}", what the person wrote is toxic:"""
prompts = [prompt.format(sentence) for sentence in sentences]

sampled_prompts = prompts

In [ ]:
sampled_prompts_tokenized = model.to_tokens(sampled_prompts, prepend_bos=False)

position_of_eos = find_bos_token(sampled_prompts_tokenized, bos_token = 50256)

pos_slice = torch.tensor(position_of_eos)

In [ ]:
y_sampled = np.array(toxicity) > 0.5
answer_tokens = ['true' if y else 'false' for y in y_sampled]
wrong_tokens = ['false' if y else 'true' for y in y_sampled]

answer_tokens = model.to_tokens(answer_tokens, prepend_bos=False)
wrong_tokens = model.to_tokens(wrong_tokens, prepend_bos=False)

In [ ]:
per_head_logit_diffs = get_heads_logit_diff(prompts = sampled_prompts_tokenized, 
                    answer_tokens = answer_tokens, 
                    wrong_tokens = wrong_tokens, 
                    position_of_eos = pos_slice,
                    model = model)

In [ ]:
imshow(
    per_head_logit_diffs,
    labels={"x":"Head", "y":"Layer"},
    title="Logit Difference From Each Head",
    width=600
)